In [31]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import re
import random

# 1. Load model dan tokenizer
model_path = "./sentiment_model"
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model.eval()

# 2. Mapping label angka ke string
label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "others"
}

# 3. Mapping emosi ke lawannya (untuk mode hibur)
opposite_emotion_map = {
    "joy": ["joy", "love"],
    "love": ["love", "joy"],
    "anger": ["joy"],
    "fear": ["joy", "love"],
    "sadness": ["joy", "love"],
    "others": ["joy", "love"]
}

# 4. Fungsi preprocessing untuk input user
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 5. Fungsi prediksi emosi dari inputan user
def predict_emotion(text):
    clean_text_input = clean_text(text)
    inputs = tokenizer(clean_text_input, return_tensors="pt", truncation=True, padding=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    label_id = torch.argmax(logits, dim=1).item()
    probs = torch.softmax(logits, dim=1).squeeze().tolist()
    return label_map[label_id], probs

# 6. Load dataset film dan musik
movies_df = pd.read_csv("E:/Semester 4/TWS/movies.csv")
music_df = pd.read_csv("E:/Semester 4/TWS/music.csv")

# 7. Pastikan kolom label dalam lowercase
movies_df['mapped_label'] = movies_df['label'].str.lower()
music_df['mapped_label'] = music_df['mapped_label'].str.lower()

# 8. Fungsi rekomendasi film
def recommend_movies(emotion_label, top_n=3):
    filtered = movies_df[movies_df['mapped_label'] == emotion_label]
    if len(filtered) >= top_n:
        sampled = filtered.sample(n=top_n, random_state=None)
    else:
        sampled = filtered
    return sampled[['title', 'overview']].to_dict(orient='records')

# 9. Fungsi rekomendasi musik
def recommend_music(emotion_label, top_n=3):
    filtered = music_df[music_df['mapped_label'] == emotion_label]
    if len(filtered) >= top_n:
        sampled = filtered.sample(n=top_n, random_state=None)
    else:
        sampled = filtered
    return sampled[['track', 'artist', 'combined_seeds']].to_dict(orient='records')

# 10. Fungsi utama rekomendasi dengan mode
def get_recommendations(user_input, mode="comfort"):
    emotion, probs = predict_emotion(user_input)
    
    if mode == "cheer":
        target_emotions = opposite_emotion_map.get(emotion, ["joy"])
        target_emotion = random.choice(target_emotions)
    else:
        target_emotion = emotion

    movies = recommend_movies(target_emotion)
    music = recommend_music(target_emotion)
    
    return {
        "input": user_input,
        "predicted_emotion": emotion,
        "mode": mode,
        "used_emotion_for_recommendation": target_emotion,
        "movies": movies,
        "music": music
    }

# 11. Contoh pemakaian
if __name__ == "__main__":
    user_text = input("What's on your mind?: ")
    mode_choice = input("I'm here for you. Should I stay with your feelings (1), or help lift your mood (2)? [1/2]: ")
    mode = "comfort" if mode_choice == "1" else "cheer"

    result = get_recommendations(user_text, mode)

    print("Detected emotion:", result["predicted_emotion"])
    print("Recommendation mode:", result["mode"])
    print("Emotion used for recommendation:", result["used_emotion_for_recommendation"])

    print("\nFilm Recommendation:")
    for movie in result["movies"]:
        print(f"- {movie['title']}: {movie['overview'][:100]}...")

    print("\nMusic Recommendation:")
    for track in result["music"]:
        print(f"- {track['track']} by {track['artist']} ({track['combined_seeds']})")

Detected emotion: sadness
Recommendation mode: comfort
Emotion used for recommendation: sadness

Film Recommendation:
- Paper Soldiers: Paper Soldiers follows an overeager burglar named Shawn (Kevin Hart) through the ups and downs of hi...
- I'm Not Ready for Christmas: Holly finds her world turned upside-down when she suddenly finds herself incapable of lying. (Don’t ...
- The Amazing Spider-Man 2: For Peter Parker, life is busy. Between taking out the bad guys as Spider-Man and spending time with...

Music Recommendation:
- Suicide Underground by Air (dark melancholy)
- The Boy Must Die by Drunk With Joy (cerebral)
- The Watch by Fatal Gift (naive)
